# 1. Import Requirements and Load date files 

In [2]:
import unicodedata, string, re, random, time, math, torch, torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import keras, numpy as np
en_input = np.load('data/processed/train_en_id_list.npy')
zh_input = np.load('data/processed/train_zh_id_list.npy')
en_embedding = np.load('data/processed/enword2vec.npy')
zh_embedding = np.load('data/processed/zhword2vec.npy')


Using TensorFlow backend.


In [3]:
print(en_input.shape)
print(zh_input.shape)
print(en_embedding.shape)
print(zh_embedding.shape)

(9903244, 30)
(9903244, 30)
(88937, 300)
(143219, 300)


# 2. Model
### Basic encoder-decoder

In [19]:
def long_t(arr): 
    return Variable(torch.from_numpy(arr))


def create_emb(emb_mat, non_trainable=False):
    output_size, emb_size = emb_mat.size()
    emb = nn.Embedding(output_size, emb_size)
    emb.load_state_dict({'weight': emb_mat})
    if non_trainable:
        for param in emb.parameters(): 
            param.requires_grad = False
    return emb, emb_size, output_size


def encode(inp, encoder):
    batch_size, input_length = inp.size()
    hidden = encoder.initHidden(batch_size).cuda()
    enc_outputs, hidden = encoder(inp, hidden)
    return long_t([SOS]*batch_size), enc_outputs, hidden    


class EncoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2):
        super(EncoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs, True)
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True, num_layers=n_layers,bidirectional=True)
        
    def forward(self, input, hidden):
        return self.gru(self.emb(input), hidden)

    def initHidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

### Adding broadcasting to Pytorch

In [5]:
def unit_prefix(x, n=1):
    for i in range(n): x = x.unsqueeze(0)
    return x

def align(x, y, start_dim=2):
    xd, yd = x.dim(), y.dim()
    if xd > yd: y = unit_prefix(y, xd - yd)
    elif yd > xd: x = unit_prefix(x, yd - xd)

    xs, ys = list(x.size()), list(y.size())
    nd = len(ys)
    for i in range(start_dim, nd):
        td = nd-i-1
        if   ys[td]==1: ys[td] = xs[td]
        elif xs[td]==1: xs[td] = ys[td]
    return x.expand(*xs), y.expand(*ys)

def aligned_op(x,y,f): return f(*align(x,y,0))

def add(x, y): return aligned_op(x, y, operator.add)
def sub(x, y): return aligned_op(x, y, operator.sub)
def mul(x, y): return aligned_op(x, y, operator.mul)
def div(x, y): return aligned_op(x, y, operator.truediv)

def dot(x, y):
    assert(1<y.dim()<5)
    x, y = align(x, y)
    
    if y.dim() == 2: return x.mm(y)
    elif y.dim() == 3: return x.bmm(y)
    else:
        xs,ys = x.size(), y.size()
        res = torch.zeros(*(xs[:-1] + (ys[-1],)))
        for i in range(xs[0]): res[i].baddbmm_(x[i], (y[i]))
        return res

### Attention Model

In [6]:
from torch.nn import parameter
def Arr(*sz): return torch.randn(sz)/math.sqrt(sz[0])

def Var(*sz): return parameter.Parameter(Arr(*sz))

In [7]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, embs, hidden_size, n_layers=2, p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.emb, emb_size, output_size = create_emb(embs)
        self.W1 = Var(hidden_size, hidden_size)
        self.W2 = Var(hidden_size, hidden_size)
        self.W3 = Var(emb_size+hidden_size, hidden_size)
        self.b2 = Var(hidden_size)
        self.b3 = Var(hidden_size)
        self.V = Var(hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=2)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, inp, hidden, enc_outputs):
        emb_inp = self.emb(inp)
        w1e = dot(enc_outputs, self.W1)
        w2h = add(dot(hidden[-1], self.W2), self.b2).unsqueeze(1)
        u = F.tanh(add(w1e, w2h))
        a = mul(self.V,u).sum(2).squeeze(2)
        a = F.softmax(a).unsqueeze(2)
        Xa = mul(a, enc_outputs).sum(1)
        res = dot(torch.cat([emb_inp, Xa.squeeze(1)], 1), self.W3)
        res = add(res, self.b3).unsqueeze(0)
        res, hidden = self.gru(res, hidden)
        res = F.log_softmax(self.out(res.squeeze(0)))
        return res, hidden

## Train

In [8]:
def train(inp, targ, encoder, decoder, enc_opt, dec_opt, crit):
    decoder_input, encoder_outputs, hidden = encode(inp, encoder)
    target_length = targ.size()[1]
    
    enc_opt.zero_grad(); dec_opt.zero_grad()
    loss = 0

    for di in range(target_length):
        decoder_output, hidden = decoder(decoder_input, hidden, encoder_outputs)
        decoder_input = targ[:, di]
        loss += crit(decoder_output, decoder_input)

    loss.backward()
    enc_opt.step(); dec_opt.step()
    return loss.data[0] / target_length

In [9]:
def req_grad_params(o):
    return (p for p in o.parameters() if p.requires_grad)

In [10]:
def trainEpochs(encoder, decoder, n_epochs, print_every=1000, lr=0.01):
    loss_total = 0 # Reset every print_every
    
    enc_opt = optim.RMSprop(req_grad_params(encoder), lr=lr)
    dec_opt = optim.RMSprop(decoder.parameters(), lr=lr)
    crit = nn.NLLLoss().cuda()
    
    for epoch in range(n_epochs):
        en, zh = get_batch(en_train, zh_train, 64)
        inp = long_t(en)
        targ = long_t(zh)
        loss = train(inp, targ, encoder, decoder, enc_opt, dec_opt, crit)
        loss_total += loss

        if epoch % print_every == print_every-1:
            print('%d %d%% %.4f' % (epoch, epoch / n_epochs * 100, loss_total / print_every))

In [11]:
from sklearn import model_selection
en_train, en_val, zh_train, zh_val = model_selection.train_test_split(
    en_input, zh_input, test_size=0.001)

In [12]:
del en_input, zh_input

In [13]:
zh_train[0], en_train[0]

(array([     0,      0,      0,      0,      0,      0,      0,      1,
         31264,      3,    169, 121817,     10,  51839,  40667,  35508,
           333,   2226,   1048,    751,     10,   1055,     39,  48110,
         13000,     10,   1817,     26,     13,      2], dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     1,
         2233, 11931, 38154,     6, 34275, 16911,   625,  1489,    55,
          899,  1039,    46,    59,    55,   563,     6,  3444,   176,
         7677,    28,     2], dtype=int32))

In [14]:
hidden_size = 128
en_emb_t = torch.FloatTensor(en_embedding)
zh_emb_t = torch.FloatTensor(zh_embedding)
encoder = EncoderRNN(en_emb_t, hidden_size)
decoder = AttnDecoderRNN(zh_emb_t, hidden_size)
del en_embedding, zh_embedding

In [20]:
def get_batch(x, y, batch_size=16):
    idxs = np.random.permutation(len(x))[:batch_size]
    return x[idxs], y[idxs]
trainEpochs(encoder, decoder, 10000, print_every=500, lr=0.005)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx